In [9]:
import os.path
import pandas as pd
import cPickle as pkl

In [7]:
homedir = r'/home/martin/git/TRSL/results'

In [8]:
runs = [
    r'updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah, initiation rates scaled by 0.01_20180221_1409_200000_ribosomes_1800s.p'
    ]

In [27]:
scales = [0.01] #, 0.1, 1.0, 10.0, 100.0]

Remove numpypy dependence:

In [11]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path, 'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

In [12]:
for run in runs:
    filepath = os.path.join(homedir, run)
    replace(filepath, "_numpypy.multiarray", "numpy.core.multiarray")

Non-proteins:

In [21]:
blacklist = ['AMP', 'ATP', 'GDP', 'GTP', 
             'peptide_bonds', 'proteins', 'ribos._bound', 'ribos._free', 'tRNA_bound', 'tRNA_free', 
             'tRNA_free_01', 'tRNA_free_02', 'tRNA_free_03', 'tRNA_free_04', 'tRNA_free_05', 'tRNA_free_06', 
             'tRNA_free_07', 'tRNA_free_08', 'tRNA_free_09', 'tRNA_free_10', 'tRNA_free_11', 'tRNA_free_12', 
             'tRNA_free_13', 'tRNA_free_14', 'tRNA_free_15', 'tRNA_free_16', 'tRNA_free_17', 'tRNA_free_18', 
             'tRNA_free_19', 'tRNA_free_20', 'tRNA_free_21', 'tRNA_free_22', 'tRNA_free_23', 'tRNA_free_24', 
             'tRNA_free_25', 'tRNA_free_27', 'tRNA_free_28', 'tRNA_free_29', 'tRNA_free_30', 'tRNA_free_31', 
             'tRNA_free_32', 'tRNA_free_33', 'tRNA_free_34', 'tRNA_free_35', 'tRNA_free_36', 'tRNA_free_37', 
             'tRNA_free_38', 'tRNA_free_39', 'tRNA_free_40', 'tRNA_free_41', 'tRNA_free_42', 'tRNA_free_43']

In [31]:
start, stop, step = 6001, 9000, 0.2

In [ ]:
for scale, run in zip(scales, runs):
    print scale
    print run
    filepath = os.path.join(homedir, run)
    results = pkl.load(open(filepath))

    print len(results['timecourses'])
    protein_synthesis_rates = {gene: (results['timecourses'][gene][-1] - results['timecourses'][gene][-1-(stop-start)])
                               / step / (stop-start) for gene in results['timecourses'] if gene not in blacklist}

0.01
updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah, initiation rates scaled by 0.01_20180221_1409_200000_ribosomes_1800s.p
